# AI Assistant For Paralysed Persons

###### Libraries required

In [1]:
import cv2
import dlib
from scipy.spatial import distance
from math import hypot
import numpy 
import pyglet
import time
import pyttsx3
import speech_recognition as sr
import pyaudio
from twilio.rest import Client
from pyfirmata import Arduino, util

#>>>>>>>>Fire Base included<<<<<<<

import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

In [2]:
# board = Arduino('COM4')

#fire base credintials

cred = credentials.Certificate('quartets-9aec7-firebase-adminsdk-vvxm2-738bf3d8dd.json')
firebase_admin.initialize_app(cred,{'databaseURL':'https://quartets-9aec7-default-rtdb.firebaseio.com'})


In [3]:
def draw_menu():
    rows, cols, _ = keyboard.shape
    th_lines = 4 # thickness lines
    cv2.line(keyboard, (int(cols/2) - int(th_lines/2), 0),(int(cols/2) - int(th_lines/2), rows),(51, 51, 51), th_lines)
    cv2.putText(keyboard, "Call & Message", (40, 300), font, 3, (255, 255, 255), 4)
    cv2.putText(keyboard, "Home Automation", (510 , 300), font, 3, (255, 255, 255), 4)
    
def eyes_contour_points(facial_landmarks):
    left_eye = []
    right_eye = []
    for n in range(36, 42):
        x = facial_landmarks.part(n).x
        y = facial_landmarks.part(n).y
        left_eye.append([x, y])
    for n in range(42, 48):
        x = facial_landmarks.part(n).x
        y = facial_landmarks.part(n).y
        right_eye.append([x, y])
    left_eye = numpy.array(left_eye, numpy.int32)
    right_eye = numpy.array(right_eye, numpy.int32)
    return left_eye, right_eye

def midpoint(p1,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)
font = cv2.FONT_HERSHEY_PLAIN

def get_blink_ratio(eye_points, facial_landmarks):
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)
    center_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4]))   
    hor_line = cv2.line(frame, left_point, right_point, (0,255,0),1)
    ver_line = cv2.line(frame, center_top, center_bottom, (0,255,0),1)
        
    #..............check length of line..............#   
    hor_line_length = hypot((left_point[0] - right_point[0]),(left_point[1] - right_point[1]))   
    ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
    ratio = (hor_line_length/ver_line_length)
    
    
    return ratio

def get_gaze_ratio(eye_points, facial_landmarks):
    left_eye_region = numpy.array([(facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y),
                                (facial_landmarks.part(eye_points[1]).x, facial_landmarks.part(eye_points[1]).y),
                                (facial_landmarks.part(eye_points[2]).x, facial_landmarks.part(eye_points[2]).y),
                                (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y),
                                (facial_landmarks.part(eye_points[4]).x, facial_landmarks.part(eye_points[4]).y),
                                (facial_landmarks.part(eye_points[5]).x, facial_landmarks.part(eye_points[5]).y)], numpy.int32)
    # cv2.polylines(frame, [left_eye_region], True, (0, 0, 255), 2)
    #print(left_eye_region)

    height, width, _ = frame.shape
    mask = numpy.zeros((height, width), numpy.uint8)
    cv2.polylines(mask, [left_eye_region], True, 255, 2)
    cv2.fillPoly(mask, [left_eye_region], 255)
    eye = cv2.bitwise_and(gray, gray, mask=mask)
    #print((height,width))

    min_x = numpy.min(left_eye_region[:, 0])
    max_x = numpy.max(left_eye_region[:, 0])
    min_y = numpy.min(left_eye_region[:, 1])
    max_y = numpy.max(left_eye_region[:, 1])

    gray_eye = eye[min_y: max_y, min_x: max_x]
    
    _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)
    height, width = threshold_eye.shape
    
    left_side_threshold = threshold_eye[0: height, 0: int(width / 2)]
    left_side_white = cv2.countNonZero(left_side_threshold)
    
    right_side_threshold = threshold_eye[0: height, int(width / 2): width]
    right_side_white = cv2.countNonZero(right_side_threshold)
    
    
    mask = cv2.resize(mask,None,fx=1/2,fy=1/2)
    gray_eye = cv2.resize(gray_eye,None,fx=5,fy=5)
    threshold_eye = cv2.resize(threshold_eye,None,fx=5,fy=5)
    left_side_threshold = cv2.resize(left_side_threshold,None,fx=5,fy=5)
    right_side_threshold = cv2.resize(right_side_threshold,None,fx=5,fy=5)
    
    if left_side_white == 0:
        gaze_ratio = 1
    elif right_side_white == 0:
        gaze_ratio = 5
    else:
        gaze_ratio =  left_side_white / right_side_white
    
    
    
    
#     cv2.putText(frame,str(gaze_ratio),(50,100),font,2,(0,0,255),3)
#     cv2.putText(frame,str(left_side_white),(50,100),font,2,(0,0,255),3)
#     cv2.putText(frame,str(right_side_white),(50,150),font,2,(0,0,255),3)
#     print((left_side_white,right_side_white,gaze_ratio))
#     cv2.imshow('left',left_side_threshold)
#     cv2.imshow('right',right_side_threshold)
#     cv2.imshow('Eye',gray_eye)
#     cv2.imshow('THershold_EYE',threshold_eye)
#     cv2.imshow("Mask",mask)
#     cv2.imshow('Mask_Eye',eye)
    

    return gaze_ratio

def gaze_position(gaze_position):
    if gaze_ratio > 3:
        #LEFT
        return 2
    elif gaze_ratio < 0.7:
        #right
        return 1       
    else:
        #CENTER
        return 0
def letter(letter_index, text,letter_light):
    
    
    if letter_index == 0:
        x = 0
        y = 0
    elif letter_index == 1:
        x = 500
        y = 0
    elif letter_index == 2:
        x = 0
        y = 100
    elif letter_index == 3:
        x = 500
        y = 100
    elif letter_index == 4:
        x = 0
        y = 200
    elif letter_index == 5:
        x = 500
        y = 200
    elif letter_index == 6:
        x = 0
        y = 300
    elif letter_index == 7:
        x = 500
        y = 300
    elif letter_index == 8:
        x = 0
        y = 400
    elif letter_index == 9:
        x = 500
        y = 400
    

    #keys
    width = 500
    height = 100
    th = 3 #thickness


    
    font_letter = cv2.FONT_HERSHEY_PLAIN
    font_scale = 3.5
    font_thick = 4

    text_size = cv2.getTextSize(text,font_letter,font_scale,font_thick)[0]
    width_text, height_text = text_size[0],text_size[1]
    #cv2.putText(keyboard,text,(100,60),font_letter,font_scale,(255,0,0),font_thick)

    #print(text_size)
    #print(width_text)
    #print(height_text)
    #..................locate alphabet in center.......................#
    text_x = int((width - width_text)/2) + x
    text_y = int((height + height_text)/2) + y
    cv2.putText(keyboard, text, (text_x,text_y), font_letter, font_scale, (0,255,255), font_thick)

    
    
    
    if letter_light is True:
        cv2.rectangle(keyboard,(x + th,y + th),(x + width - th,y + height - th),(0,255,255),-1)
        cv2.putText(keyboard, text, (text_x,text_y), font_letter, font_scale, (0, 0, 0), 5)
        

    else:
        cv2.rectangle(keyboard,(x + th,y + th),(x + width - th,y + height - th),(0,255,255),th)
        


In [4]:
###Voice Assistant###
engine=pyttsx3.init('sapi5')
voices=engine.getProperty('voices')

engine.setProperty('voice',voices[0].id)

def speak(audio):
    engine.say(audio)
    engine.runAndWait()
    
    
def takecommand():
    r=sr.Recognizer()
   
        
    with sr.Microphone() as source:
        print("listerning......")
        r.pause_threshold=1
        r.adjust_for_ambient_noise(source,duration=1)
        audio=r.listen(source)
     
        
    try:
        print("Wait for few Moments")
        query=r.recognize_google(audio,language='en-in')
        print("User Said:",query)
            
        
        
        
    except Exception as e:
        print(e)
        #speak("Say that again please")

        
        return "None"
    
    return query


In [5]:
def phone_call(response,from_num,To_num):
    account_sid="ACbe10fe9d09b294b8bd3655e65f8a472d"
    auth_token="e18873a8a786a291ef31d2f092238591"

    client=Client(account_sid,auth_token)

    call=client.calls.create(twiml=response,
                             to=To_num,
                            from_=from_num)

def send_message(message_body,from_num,To_num):

    account_sid="ACbe10fe9d09b294b8bd3655e65f8a472d"
    auth_token="e18873a8a786a291ef31d2f092238591"


    client=Client(account_sid,auth_token)

    message = client.messages.create(body=message_body,from_=from_num,to=To_num)

    print (message.sid)

In [ ]:
#play sound
sound = pyglet.media.load("zapsplat_cartoon_bubble_pop_003_40275.mp3",streaming=False)

cap=cv2.VideoCapture(0)
cap2=cv2.VideoCapture(1)#for video recording iriun
board1 =  numpy.zeros((500,500),numpy.uint8)
board1[:] = 255


detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")


#Condition Settings
keyboard = numpy.zeros((500,1000,3),numpy.uint8)

key_set_1 = {0:"Rest Room", 1:"Food & Water", 2:"Take Me Out", 3:"Medication", 4:"Bed Shifting",
            5:"Rest Room", 6:"Food & Water", 7:"Take Me Out", 8:"Medication", 9:"Bed Shifting"}

key_set_2 = {0:"Red ON", 1:"Red Off", 2:"Blue ON", 3:"Blue Off", 4:"Green ON", 5:"Green Off", 6:"Yellow ON",
             7:"Yellow Off", 8:"Servo ON", 9:"Servo Off" }


########DELAY STATUS######
C_MSG = 0 #CALL & MESSAGE
H_AUTO = 0 #HOME AUTOMATION
GREET = 0 #GREETING
status = ""
color = (0,0,0)

#Counters
frames = 0
letter_index = 0
blinking_frames = 0
frame_to_blink = 6
frames_active_letters = 30

#Settings
text = ""
keyboard_selected = "Call & Message"
last_keyboard_selected = "Call & Message"
select_keyboard_menu = True
keyboard_selection_frames = 0


#......................Phone Numbers From and To..............#

from_num = '+18642147085'
To_num = '+916383531566'

#..............Fire base .........................#
reference = db.reference("IRIS/")
home_automation_ref = reference.child('HomeAutomation')
appliance = ['appliance1','appliance2','appliance3','appliance4']



while True:
    _, frame=cap.read() 
    _, mobile=cap2.read() #for video recording 
    frame1 = frame
    frame =  cv2.resize(frame,None,fx=0.5,fy=0.5)
    mobile =  cv2.resize(mobile,None,fx=0.5,fy=0.5) #for video recording
    frame1 =  cv2.resize(frame,None,fx=0.5,fy=0.5)
    keyboard[:] = (0,0,0)
    rows, cols,_ = frame.shape
    
    frames = frames + 1
    dir_frame = numpy.zeros((500,500,3), numpy.uint8)
    
    #..........Face Detecion...........#
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    #......draw a white space for loading bar.......#
    frame[rows - 50: rows, 0: cols] = (255,255,255)
    
    if select_keyboard_menu is True:
         draw_menu()
    
    
    if keyboard_selected == "Call & Message":
        key_set = key_set_1
    else:
        key_set = key_set_2
    active_letter = key_set[letter_index]
    
    
    faces=detector(gray)    
    for face in faces:
        #print (face)
        #.........To Indicate Green rectangle over face...........#
        x1=face.left()
        y1=face.top()
        x2=face.right()
        y2=face.bottom()
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
        landmarks = predictor(gray,face)
        left_eye, right_eye = eyes_contour_points(landmarks)

        
        #..............Detect Blinking...............#
        left_eye_ratio = get_blink_ratio([36,37,38,39,40,41], landmarks)
        right_eye_ratio = get_blink_ratio([42,43,44,45,46,47], landmarks)
        blink_ratio = (left_eye_ratio + right_eye_ratio)/2 
        #print(blink_ratio)
        
        #......Eye color.....#
        cv2.polylines(frame, [left_eye],True, (0,0,255),2)
        cv2.polylines(frame, [right_eye],True, (0,0,255),2)
        
        if select_keyboard_menu is True:
            gaze_ratio_left_eye = get_gaze_ratio([36,37,38,39,40,41], landmarks)
            gaze_ratio_right_eye = get_gaze_ratio([42,43,44,45,46,47], landmarks)
            gaze_ratio = (gaze_ratio_right_eye + gaze_ratio_left_eye)/2
            gaze_ratio = round(gaze_ratio,2)
            cv2.putText(frame,str(gaze_ratio),(50,100),font,2,(255,0,255),3)
            
            #.......GAZE POSITION.........#  
            gaze_Position = gaze_position(gaze_ratio)
            if gaze_Position == 2:
                C_MSG = C_MSG + 1 
                H_AUTO = 0 
                GREET = 0
                if C_MSG > 10:
                    status = "Call & Message"
                    color = (0,255,255)
                    cv2.putText(frame,status,(50,150),font,2,color,3)
                    dir_frame[:] = (0,255,0)
                    keyboard_selected = "Call & Message"
                    keyboard_selection_frames += 1
                    # If Kept gaze on one side more than 15frames, move to keyboard
                    if keyboard_selection_frames == 15:
                        select_keyboard_menu = False
                        #speak("Callllllll Anddd Messagesssss")
                        frames = 0
                        keyboard_selection_frames = 0
                        
                    if keyboard_selected != last_keyboard_selected:
                        last_keyboard_selected = keyboard_selected
                        keyboard_selection_frames = 0

            
        
                
            if gaze_Position == 1:
                C_MSG = 0 
                H_AUTO = H_AUTO + 1 
                GREET = 0
                if H_AUTO > 10:
                    status = "Home Automation"
                    color = (0,255,255)
                    cv2.putText(frame,status,(50,150),font,2,color,3)
                    dir_frame[:] = (255,0,255)
                    keyboard_selected = "Home Automation"
                    keyboard_selection_frames += 1
                    # If Kept gaze on one side more than 15frames, move to keyboard
                    if keyboard_selection_frames == 15:
                        select_keyboard_menu = False
                        #speak("Home Automationnn")
                        frames = 0
                        keyboard_selection_frames = 0
                        
                    if keyboard_selected != last_keyboard_selected:
                        last_keyboard_selected = keyboard_selected
                        keyboard_selection_frames = 0


            if gaze_Position != 1 and gaze_Position != 2:
                C_MSG = 0 
                H_AUTO = 0
                GREET = GREET + 1
                if GREET > 5:
                    status = "Hi..!"
                    color = (0,255,255)
                    cv2.putText(frame,status,(50,150),font,2,color,3)
        
        
        else:
            # Detect the blinking to select the key that is lighting up
            if blink_ratio > 5.5:
                #cv2.putText(frame,'Blinking', (50,100), font, 3, (255,255,0),thickness = 3)
                blinking_frames = blinking_frames + 1
                frames = frames - 1
                
                #show green when eyes closed
                cv2.polylines(frame, [left_eye],True, (0,255,0),2)
                cv2.polylines(frame, [right_eye],True, (0,255,0),2)
                

                #typing letter
                if blinking_frames == frame_to_blink:
                    if active_letter == "Rest Room":
                        text += "Rest Room"
                        response = ('<Response><Say>Hello.......This is a emergency call Take me to Rest Room........</Say></Response>')
                        phone_call(response,from_num,To_num)
                        sound.play()
                        time.sleep(10)
                        
                        message_body = "Emergency....\n Take me to Rest Room ...\n Thank You..:)"
                        #send_message(message_body,from_num,To_num)
                        time.sleep(5)
                        
                    if active_letter == "Food & Water":
                        text += "Food & Water"
                        response = ("<Response><Say>Hello.......I'm hungry Give me Food or else Water........</Say></Response>")
                        phone_call(response,from_num,To_num)
                        sound.play()
                        time.sleep(10)
                        
                        message_body = "Emergency....\nI'm hungry Give me Food or else Water ...\n Thank You..:)"
                        #send_message(message_body,from_num,To_num)
                        time.sleep(5)
                    if active_letter == "Take Me Out":
                        text += "Take Me Out"
                        response = ("<Response><Say>Hello.......I'm bored Take Me Out........</Say></Response>")
                        phone_call(response,from_num,To_num)
                        sound.play()
                        time.sleep(10)
                        
                        message_body = "Emergency....\nI'm bored Take Me Out ...\n Thank You..:)"
                        #send_message(message_body,from_num,To_num)
                        time.sleep(5)
                    if active_letter == "Medication":
                        text += "Medication"
                        response = ("<Response><Say>Hello.......It's time for my medication........</Say></Response>")
                        phone_call(response,from_num,To_num)
                        sound.play()
                        time.sleep(10)
                        
                        message_body = "Emergency....\nIt's time for my medication ...\n Thank You..:)"
                        #send_message(message_body,from_num,To_num)
                        time.sleep(5)
                    if active_letter == "Bed Shifting":
                        text += "Bed Shifting"
                        response = ("<Response><Say>Hello.......I'm not comfortable, kindly shift my bed........</Say></Response>")
                        phone_call(response,from_num,To_num)
                        sound.play()
                        time.sleep(10)
                        
                        message_body = "Emergency....\nI'm not comfortable, kindly shift my bed ...\n Thank You..:)"
                        #send_message(message_body,from_num,To_num)
                        time.sleep(5)
                    if active_letter == "Red ON":
                        text += "Red ON"
                        # appliance status ON/OFF
                        status = 1
                        home_automation_ref.set({appliance[0]: status})
                        
                        #time.sleep(10)
                    if active_letter == "Red Off":
                        text += "Red Off"
                        # appliance status ON/OFF
                        status = 0
                        home_automation_ref.set({appliance[0]: status})
                        #time.sleep(10)
                        
                    if active_letter == "Blue ON":
                        text += "Blue ON"
                        # appliance status ON/OFF
                        status = 1
                        home_automation_ref.set({appliance[1]: status})
                        
                       # time.sleep(10)
                    if active_letter == "Blue Off":
                        text += "Blue Off"
                        # appliance status ON/OFF
                        status = 0
                        home_automation_ref.set({appliance[1]: status})
                       
                        #time.sleep(10)
                    if active_letter == "Green ON":
                        text += "Green ON"
                        # appliance status ON/OFF
                        status = 1
                        home_automation_ref.set({appliance[2]: status})
                        #time.sleep(10)
                    if active_letter == "Green Off":
                        text += "Green Off"
                        # appliance status ON/OFF
                        status = 0
                        home_automation_ref.set({appliance[2]: status})
                        #time.sleep(10)
                    if active_letter == "Yellow ON":
                        text += "Yellow ON"
                        # appliance status ON/OFF
                        status = 1
                        home_automation_ref.set({appliance[3]: status})
                        #time.sleep(10)
                    if active_letter == "Yellow Off":
                        text += "Yellow Off"
                        # appliance status ON/OFF
                        status = 0
                        home_automation_ref.set({appliance[3]: status})
                        #time.sleep(10)
                    if active_letter == "Servo ON":
                        text += "Servo ON"
                        #board.digital[10].write(180)
                        #time.sleep(10)
                    if active_letter == "Servo Off":
                        text += "Servo Off"
                        #board.digital[10].write(0) 
                        #time.sleep(10)
                        
                    select_keyboard_menu = True
            else:
                blinking_frames = 0
                
    
    if select_keyboard_menu is False:
        if frames == frames_active_letters:
            letter_index = letter_index + 1
            frames = 0
        if letter_index == 10:
            letter_index = 0
        
        for i in range(10):
            if i == letter_index:
                light = True
            else:
                light = False
            letter(i,key_set[i],light)
            
    cv2.putText(board1, text,(10,100), font , 4, 0, 3)
    
    # loading bar
    percentage_blinking = blinking_frames / frame_to_blink
    loading_x = int(cols * percentage_blinking)
    cv2.rectangle(frame,(0, rows -50),(loading_x,rows),(51,51,51),-1)
    
        
    
                

        
        #print((gaze_ratio_left_eye,gaze_ratio_right_eye))
    
    cv2.imshow("Face ",frame)
    #cv2.imshow('Direction',dir_frame)
    cv2.imshow('Conditions', keyboard)
    #cv2.imshow("Home Automation", keyboard)
    #cv2.imshow("Board", board)
    k=cv2.waitKey(1)
    if k==ord('q'):
        break

cap.release() 
cv2.destroyAllWindows()
